# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,escuintla,14.3050,-90.7850,26.43,97,100,0.71,GT,1727420920
1,1,okha,53.5739,142.9478,11.38,75,100,3.79,RU,1727420921
2,2,ola,59.5833,151.2833,7.03,85,57,3.14,RU,1727420922
3,3,hawaiian paradise park,19.5933,-154.9731,22.73,96,100,2.06,US,1727420924
4,4,walvis bay,-22.9575,14.5053,14.49,75,85,4.08,NaN,1727420925


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,nili,33.7218,66.1302,24.33,13,0,1.47,AF,1727420967
60,60,fomboni,-12.2800,43.7425,25.03,70,0,1.52,KM,1727420999
61,61,remire-montjoly,4.9167,-52.2667,23.02,100,0,0.00,GF,1727421000
142,142,itaborai,-22.7444,-42.8594,24.78,54,0,1.98,BR,1727421113
188,188,hanting,36.7708,119.2108,26.56,36,0,2.47,CN,1727420967
200,200,cassano allo ionio,39.7830,16.3182,24.22,53,0,2.81,IT,1727421192
215,215,korinthos,37.9407,22.9573,24.38,50,0,2.10,GR,1727421151
228,228,beipiao,41.7919,120.7792,22.60,47,0,4.41,CN,1727421007
304,304,crane,31.3974,-102.3501,21.80,47,0,4.03,US,1727421343
338,338,awjilah,29.1081,21.2869,25.88,56,0,4.25,LY,1727421390


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]


hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,nili,AF,33.7218,66.1302,13,
60,fomboni,KM,-12.2800,43.7425,70,
61,remire-montjoly,GF,4.9167,-52.2667,100,
142,itaborai,BR,-22.7444,-42.8594,54,
188,hanting,CN,36.7708,119.2108,36,
200,cassano allo ionio,IT,39.7830,16.3182,53,
215,korinthos,GR,37.9407,22.9573,50,
228,beipiao,CN,41.7919,120.7792,47,
304,crane,US,31.3974,-102.3501,47,
338,awjilah,LY,29.1081,21.2869,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nili - nearest hotel: No hotel found
fomboni - nearest hotel: Mohéli La Plage
remire-montjoly - nearest hotel: Complexe Belova
itaborai - nearest hotel: No hotel found
hanting - nearest hotel: 山东高速服务区管理有限公司青银路潍坊分公司高速宾馆
cassano allo ionio - nearest hotel: B&B II Comignolo did Sofia
korinthos - nearest hotel: Apollon
beipiao - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel
mao - nearest hotel: Hevresac
baharly - nearest hotel: No hotel found
argo - nearest hotel: Apollon Hotel
al burayqah - nearest hotel: فندق ماس ليبيا
isfahan - nearest hotel: هتل فیروزه
prado - nearest hotel: Pousada Recanto do Prado


,City,Country,Lat,Lng,Humidity,Hotel Name
36,nili,AF,33.7218,66.1302,13,No hotel found
60,fomboni,KM,-12.2800,43.7425,70,Mohéli La Plage
61,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
142,itaborai,BR,-22.7444,-42.8594,54,No hotel found
188,hanting,CN,36.7708,119.2108,36,山东高速服务区管理有限公司青银路潍坊分公司高速宾馆
200,cassano allo ionio,IT,39.7830,16.3182,53,B&B II Comignolo did Sofia
215,korinthos,GR,37.9407,22.9573,50,Apollon
228,beipiao,CN,41.7919,120.7792,47,No hotel found
304,crane,US,31.3974,-102.3501,47,No hotel found
338,awjilah,LY,29.1081,21.2869,56,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.65,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   frame_width = 800,
                                   frame_height = 600,
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)